In [1]:
import pandas as pd
import numpy as np

In [2]:
workers_answers = pd.read_csv("Crowd_Frame riscaricato/data/result/task1/Dataframe/workers_answers.csv")
workers_questionnaire = pd.read_csv("Crowd_Frame riscaricato/data/result/task1/Dataframe/workers_questionnaire.csv")

In [3]:
workers_answers

,worker_id,paid,task_name,batch_name,unit_id,try_last,try_current,action,time_submit,time_submit_parsed,...,time_start,time_end,time_elapsed,time_start_parsed,time_end_parsed,accesses,global_outcome,global_form_validity,gold_checks,time_spent_check
0,7GFH0CXPHPTSPX,True,task1,batch1,unit_1,1,1,Next,"Tue, 16 Jan 2024 09:38:22 GMT",2024-01-16 09:38:22 00:00,...,1.705398e+09,1.705398e+09,137.07,2024-01-16 09:36:05,2024-01-16 09:38:22,1,True,True,True,True
1,7GFH0CXPHPTSPX,True,task1,batch1,unit_1,1,1,Next,"Tue, 16 Jan 2024 09:41:43 GMT",2024-01-16 09:41:43 00:00,...,1.705398e+09,1.705398e+09,201.00,2024-01-16 09:38:22,2024-01-16 09:41:43,1,True,True,True,True
2,7GFH0CXPHPTSPX,True,task1,batch1,unit_1,1,1,Next,"Tue, 16 Jan 2024 09:43:01 GMT",2024-01-16 09:43:01 00:00,...,1.705398e+09,1.705398e+09,78.06,2024-01-16 09:41:43,2024-01-16 09:43:01,1,True,True,True,True
3,7GFH0CXPHPTSPX,True,task1,batch1,unit_1,1,1,Next,"Tue, 16 Jan 2024 09:44:45 GMT",2024-01-16 09:44:45 00:00,...,1.705398e+09,1.705398e+09,104.07,2024-01-16 09:43:01,2024-01-16 09:44:45,1,True,True,True,True
4,7GFH0CXPHPTSPX,True,task1,batch1,unit_1,1,1,Finish,"Tue, 16 Jan 2024 09:45:59 GMT",2024-01-16 09:45:59 00:00,...,1.705398e+09,1.705398e+09,73.03,2024-01-16 09:44:45,2024-01-16 09:45:58,1,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,T4MEORG9STZITP,True,task1,batch1,unit_10,1,1,Back,"Tue, 16 Jan 2024 18:25:12 GMT",2024-01-16 18:25:12 00:00,...,1.705429e+09,1.705430e+09,125.46,2024-01-16 18:23:06,2024-01-16 18:25:12,1,True,True,True,True
83,T4MEORG9STZITP,True,task1,batch1,unit_10,1,1,Next,"Tue, 16 Jan 2024 18:25:16 GMT",2024-01-16 18:25:16 00:00,...,1.705429e+09,1.705429e+09,139.79,2024-01-16 18:20:46,2024-01-16 18:22:51,3,True,True,True,True
84,T4MEORG9STZITP,True,task1,batch1,unit_10,1,1,Next,"Tue, 16 Jan 2024 18:25:17 GMT",2024-01-16 18:25:17 00:00,...,1.705429e+09,1.705430e+09,126.40,2024-01-16 18:23:06,2024-01-16 18:25:12,2,True,True,True,True
85,T4MEORG9STZITP,True,task1,batch1,unit_10,1,1,Next,"Tue, 16 Jan 2024 18:27:16 GMT",2024-01-16 18:27:16 00:00,...,1.705430e+09,1.705430e+09,119.69,2024-01-16 18:25:17,2024-01-16 18:27:16,1,True,True,True,True


In [4]:
absolute_frequencies = workers_questionnaire['question_answer_text'].value_counts()
relative_frequencies = absolute_frequencies / len(workers_questionnaire)

average_confidence = workers_answers.groupby('document_index')['Confidence_value'].mean()
average_time_elapsed = workers_answers.groupby('document_index')['time_elapsed'].mean()

In [5]:
#da verificare
total_units = len(workers_answers)
matching_units = workers_answers[workers_answers['Reliability_value'] == workers_answers['Reliability_value']].shape[0]

percent_agreement = (matching_units / total_units) * 100

In [6]:
# Inizializza una lista per i risultati
result_list = []

# Cicla sui valori unici della colonna document_id
for doc_id in workers_answers['document_id'].unique():
    # Filtra il DataFrame per il documento corrente
    doc_df = workers_answers[workers_answers['document_id'] == doc_id]

    # Conserva solo la prima risposta per ogni worker_id
    unique_worker_ids = doc_df['worker_id'].unique()
    for worker_id in unique_worker_ids:
        first_answer_idx = doc_df[doc_df['worker_id'] == worker_id].index[0]
        first_answer_row = doc_df.loc[first_answer_idx]
        result_list.append({'document_id': doc_id, 'Neutrality_index': first_answer_row['Neutrality_index']})

# Crea un DataFrame dai risultati
result_df = pd.DataFrame(result_list)
#print(result_df)# Visualizza il DataFrame risultante
print(result_df)

# Raggruppa per document_id e calcola la percentuale di accordo
grouped_df = result_df.groupby('document_id')['Neutrality_index'].value_counts(normalize=True).mul(100).unstack().fillna(0)

# Visualizza il DataFrame risultante
#print(grouped_df)

   document_id  Neutrality_index
0   B015XC3B46               3.0
1   B015XC3B46               1.0
2   B015XC3B46               1.0
3   B015XC3B46               3.0
4   B0002KMHD6               2.0
..         ...               ...
65  B00QTSR5GO               4.0
66  B00QTSR5GO               3.0
67  B00QTSR5GO               4.0
68  B00QTSR5GO               3.0
69  B00QTSR5GO               2.0

[70 rows x 2 columns]


In [7]:
grouped_df = result_df.groupby(['document_id', 'Neutrality_index']).size().reset_index(name='counts')

# Crea un DataFrame pivot per avere le Neutrality_description come colonne
pivot_df = grouped_df.pivot(index='document_id', columns='Neutrality_index', values='counts').fillna(0)

# Calcola la percentuale di accordo per ciascun document_id
total_documents = pivot_df.shape[0]
matching_values = pivot_df.apply(lambda row: row.max(), axis=1)
percent_agreement = (matching_values / total_documents) * 100

# Visualizza il DataFrame risultante con la percentuale di accordo per ciascun document_id
result_percent_df = pd.DataFrame({'document_id': percent_agreement.index, 'Percent_Agreement': percent_agreement.values})

print(result_percent_df)

   document_id  Percent_Agreement
0   B00004SVUZ          20.000000
1   B00006JN5L          13.333333
2   B0001DHQPI          13.333333
3   B0002KMHD6          26.666667
4   B000O39U3Q          13.333333
5   B001DYVHY0          13.333333
6   B0029MAXR2          13.333333
7   B003TO59D2          20.000000
8   B0041RUM0Q          13.333333
9   B0050SYWGU          13.333333
10  B005ZNWT4C          20.000000
11  B00PNVOBTC          13.333333
12  B00QTSR5GO          13.333333
13  B015XC3B46          13.333333
14  B01G6LXXWI           6.666667


In [8]:
grouped_df = result_df.groupby(['document_id', 'Neutrality_index']).size().reset_index(name='counts')

# Crea un DataFrame pivot per avere le Neutrality_description come colonne
pivot_df = grouped_df.pivot(index='document_id', columns='Neutrality_index', values='counts').fillna(0)

# Calcola il pairwise agreement per ciascun document_id
total_documents = pivot_df.shape[0]
pairwise_agreement = 2 * pivot_df.apply(lambda row: row.max(), axis=1) / (total_documents * (total_documents - 1))

# Visualizza il DataFrame risultante con il pairwise agreement per ciascun document_id
result_pairwise_df = pd.DataFrame({'document_id': pairwise_agreement.index, 'Pairwise_Agreement': pairwise_agreement.values})

print(result_pairwise_df)

   document_id  Pairwise_Agreement
0   B00004SVUZ            0.028571
1   B00006JN5L            0.019048
2   B0001DHQPI            0.019048
3   B0002KMHD6            0.038095
4   B000O39U3Q            0.019048
5   B001DYVHY0            0.019048
6   B0029MAXR2            0.019048
7   B003TO59D2            0.028571
8   B0041RUM0Q            0.019048
9   B0050SYWGU            0.019048
10  B005ZNWT4C            0.028571
11  B00PNVOBTC            0.019048
12  B00QTSR5GO            0.019048
13  B015XC3B46            0.019048
14  B01G6LXXWI            0.009524


In [29]:
# Raggruppa per document_id e conta le occorrenze di ciascun valore Neutrality_description
grouped_df = result_df.groupby(['document_id', 'Neutrality_index']).size().reset_index(name='counts')

# Crea un DataFrame pivot per avere le Neutrality_description come colonne
pivot_df = grouped_df.pivot(index='document_id', columns='Neutrality_index', values='counts').fillna(0)

# Aggiungi una colonna con la somma totale delle risposte per ciascun document_id
pivot_df['Total'] = pivot_df.sum(axis=1)



# Visualizza il DataFrame risultante
print(pivot_df)

Neutrality_index,0.0,1.0,2.0,3.0,4.0,Total
document_id,,,,,,
B00004SVUZ,0.0,1.0,1.0,3.0,0.0,5.0
B00006JN5L,2.0,0.0,2.0,0.0,1.0,5.0
B0001DHQPI,1.0,2.0,0.0,2.0,0.0,5.0
B0002KMHD6,0.0,0.0,1.0,4.0,0.0,5.0
B000O39U3Q,1.0,1.0,0.0,2.0,1.0,5.0
B001DYVHY0,0.0,1.0,0.0,2.0,2.0,5.0
B0029MAXR2,0.0,2.0,1.0,2.0,0.0,5.0
B003TO59D2,3.0,2.0,0.0,0.0,0.0,5.0
B0041RUM0Q,0.0,0.0,0.0,2.0,2.0,4.0


In [30]:
somma = 0
lista = []
for document_id in pivot_df:
    for j in pivot_df[document_id]:
        somma += j*(j-1)

    lista[document_id]
result = somma /20

print(result)


Neutrality_index
2.0        2.0
3.0       36.0
4.0        6.0
Total    260.0
dtype: float64

In [ ]:
import numpy as np

def fairness(dataset):
    # Calculate the probability of each document being in each neutrality class
    neutrality_counts = np.zeros([5, 5])
    for document in dataset:
        neutrality_index = document[0]
        count = document[1 + neutrality_index]
        neutrality_counts[neutrality_index, count - 1] += 1
    probabilities = neutrality_counts / len(dataset)

    # Calculate the probability of each user being in each neutrality class
    user_counts = np.zeros([5, 5])
    for document in dataset:
        document_id = document[2]
        neutrality_index = document[0]
        count = document[1 + neutrality_index]
        user_counts[document_id - 1, count - 1] += 1
    user_probabilities = user_counts / len(dataset)

    # Calculate the fairness for each user
    fairness = []
    for user_id in range(5):
        user_p = user_probabilities[user_id]
        fairness_for_user = 0
        for neutrality_index in range(5):
            p_ij = probabilities[neutrality_index, 0] * user_probabilities[user_id, 0]
            for count in range(1, 4):
                p_ij += probabilities[neutrality_index, count] * user_probabilities[user_id, count]
            fairness_for_user += user_p * p_ij
        fairness.append(fairness_for_user)

    # Average the fairness scores for all users
    fairness_average = sum(fairness) / len(fairness)

    return fairness_average


# Load the dataset

with open('pivot_df.csv') as f:
    reader = csv.reader(f)
    for row in reader:
        dataset.append(row)

# Calculate the fairness of the dataset
fairness_average = fairness(dataset)

print(fairness_average)